# Validación Cruzada

📌 La validación del modelo forma parte de las ultimas etapas del proceso de Machine Learning, una vez entrenado el modelo, éste debe evaluarse para saber que tan bueno es, si sus predicciones son de calidad o si esta sesgado.

**Tipos de validación**

**Hold-on** <br>
Dividir los datos en entrenamiento (80%) y pruebas (20%).

**Usarla cuando**
- Se requiere un prototipo rápido.
- No se tiene mucho conocimiento (principiante).
- No se tiene suficiente poder de cómputo para utilizar otro tipo de validación.

📌 **NOTA**: Hold-on ha sido el método de validación en los modelos anteriores.

<hr />

**Validación cruzada** (cross-validation) <br>

🎯 Asegurar que todos los puntos de datos tengan la oportunidad de pertenecer al conjunto de prueba, y promediar el rendimiento en multiples “pliegues” (folds). 

- `K-Folds`: Divide los datos en `K` subconjuntos del mismo tamaño.
    - Se itera `K` veces, y en cada iteración se selecciona un pliegue para el conjunto de pruebas (validación).
    - Los `K-1` pliegues restantes se utilizan para entrenar al modelo.
    - Al final de promedian las métricas de los entrenamiento en `K` iteraciones.
    
    **Usarla cuando**
    
    - Se requiere integrar con técnicas de optimización parámetrica.
    - Se cuenta con más tiempo para realizar las pruebas.
    - Se cuenta con el equipo de computo suficiente para realizar desarrollar ML.

<div style="margin-left: 50px">
	<img src="../imgs/cv-kfolds.png" />
</div>

- `LOOCV`(Leave-One-Out cross validation): Caso extremo de `K-Folds`, donde `K` es igual al número total de muestras del dataset (`N`).
💡 Esto quiere decir que si el dataset tiene 10,000 filas, `LOOCV` tendrá 10,000 iteraciones = 10,000 entrenamientos.
    - En cada iteración se toma una única muestra como conjunto de pruebas (validación).
    - Las `N-1` muestras se utilizan como conjunto de entrenamiento.
    - El proceso se repite `N` veces.
    - El rendimiento final se promedia sobre las `N` iteraciones.
    
    **Usarla cuando**    
    - Se tiene gran poder de cómputo.
    - Se cuentan con pocos datos para dividirlos en test/train.

<div style="margin-left: 50px">
	<img src="../imgs/cv-loocv.png" />
</div>

## Implementación

Librerias

In [1]:
import pandas as pd
import numpy as np

# Modelo 
from sklearn.tree import DecisionTreeRegressor

# Validación cruzada
from sklearn.model_selection import cross_val_score, KFold, LeaveOneOut

In [2]:
%run 0.0-ml_professional-setup.ipynb

Carga del dataset `World Happiness` (2017).

In [3]:
file_path = path.data_raw_dir("2017.csv")
df = pd.read_csv(file_path)
df.columns = ["country","rank","score","high","low","gdp","family","lifexp","freedom","generosity","corruption","dystopia"]
df.head()

,country,rank,score,high,low,gdp,family,lifexp,freedom,generosity,corruption,dystopia
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


Quitamos las variables categorícas y creamos un dataset con los features y otro con los targets.

In [4]:
X = df.drop(columns=["country", "score"], axis=1)
y = df["score"]

print(f"X: {X.shape}")
print(f"y: {y.shape}")

X: (155, 10)
y: (155,)


### K-Folds 

In [5]:
# Instancia del modelo 
model_kf = DecisionTreeRegressor()

# Aplicamos validación cruzada con 5 folds
scores = cross_val_score(model_kf, X, y, cv=5, scoring="neg_mean_squared_error")
print(np.abs(np.mean(scores)))

0.20104697347549302


Utilizamos la instancia de `KFolds` como el método de validación para el modelo.

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model_kf, X, y, cv=kf, scoring="neg_mean_squared_error")

print(np.abs(np.mean(scores)) )

0.004446385725487936


Al utilizar la instancia de `KFold` en el parámetro `cv` como cross-validation strategy, en lugar de solo indicarle el número de folds, obtenemos un mejor rendimiento del modelo.

El método `split` de la clase `KFold` nos devuelve los "folds" con los indices de los datos con que se entrena y prueba el modelo.

In [7]:
for train, test in kf.split(X, y):
  print(f"train:\n{train}\n")
  print(f"test:\n{test}\n")
  print("="*60)

train:
[  0   1   2   3   4   5   6   7   8  10  11  13  14  16  17  20  21  22
  23  25  26  27  28  32  33  34  35  36  37  38  39  40  41  43  44  46
  47  48  49  50  51  52  53  54  55  56  57  58  59  61  62  63  64  65
  66  67  70  71  72  73  74  76  77  79  80  82  83  85  87  88  89  91
  92  94  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111
 112 113 114 115 116 119 120 121 122 123 124 125 127 128 129 130 132 133
 134 135 136 139 140 141 143 144 145 146 148 149 150 151 152 154]

test:
[  9  12  15  18  19  24  29  30  31  42  45  60  68  69  75  78  81  84
  86  90  93  95 117 118 126 131 137 138 142 147 153]

train:
[  0   1   2   3   5   6   7   8   9  12  13  14  15  17  18  19  20  21
  23  24  25  28  29  30  31  33  34  35  37  38  39  40  42  43  44  45
  46  47  48  49  50  52  53  54  57  58  59  60  61  62  63  64  68  69
  70  71  72  73  74  75  77  78  80  81  83  84  86  87  88  89  90  91
  92  93  94  95  97  98  99 100 101 102 103 104 105 10